#**Step 1: Install All the Required Packages**

In [43]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
pip install datasets

#**Step 2: Import All the Required Libraries**

In [44]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

#**Step 3: Define hyperparameters**

In [45]:
# LLAMA Model to use
model_name = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 32

# Dropout probability for LoRA layers
lora_dropout = 0.1

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False


# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 3

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 8

# Batch size per GPU for evaluation
per_device_eval_batch_size = 8

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 5e-5

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

#**Step 4:Create model and fine tune**

In [47]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
train_df = pd.read_parquet("hf://datasets/mediabiasgroup/BABE/" + splits["train"])
test_df = pd.read_parquet("hf://datasets/mediabiasgroup/BABE/" + splits["test"])
train_df = train_df.head(200)
test_df = test_df.head(200)
train_df = train_df.dropna(subset=["type", "label"])  # Drop rows where "type" or "label" is NaN
test_df = test_df.dropna(subset=["type", "label"])  # Drop rows where "type" or "label" is NaN
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df["type"] = train_df["type"].astype(str)
test_df["type"] = test_df["type"].astype(str)

print(train_df.shape)  # Ensure all columns have the same number of rows
print(train_df.iloc[55])  # Check row 56 if it’s throwing the error

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

def preprocess_function(examples):
  print(train_df["type"].head())  # Inspect the first few rows of the type column
  # print("----", examples["type"].dtype)
  # Ensure the input is properly formatted
  print(train_df.columns)  # Should include "type" and "label"

  inputs = [str(t) for t in examples["text"]]  # Replace "type" with your column name
  return tokenizer(
      inputs,
      truncation=True,
      padding="max_length",
      max_length=512,
  )



# Tokenize datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

train_dataset = train_dataset.map(lambda x: {"label": train_df["label"]}, batched=True)
test_dataset = test_dataset.map(lambda x: {"label": test_df["label"]}, batched=True)

# Format datasets for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

(130, 9)
text             People were tear-gassed, manhandled, shoved, b...
news_link        https://www.huffpost.com/entry/pentagon-trump-...
label                                                            1
outlet                                                    HuffPost
topic                                             trump-presidency
type                                                          left
label_opinion                Somewhat factual but also opinionated
biased_words     ['tear-gassed', 'manhandled', 'shoved', 'batte...
uuid                                        YSJtZuwfuSeZuig9paqZKf
Name: 55, dtype: object


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/130 [00:00<?, ? examples/s]

0      left
1      left
2    center
3     right
4     right
Name: type, dtype: object
Index(['text', 'news_link', 'label', 'outlet', 'topic', 'type',
       'label_opinion', 'biased_words', 'uuid'],
      dtype='object')


Map:   0%|          | 0/129 [00:00<?, ? examples/s]

0      left
1      left
2    center
3     right
4     right
Name: type, dtype: object
Index(['text', 'news_link', 'label', 'outlet', 'topic', 'type',
       'label_opinion', 'biased_words', 'uuid'],
      dtype='object')


Map:   0%|          | 0/130 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

In [48]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Sources:
# 1. https://scikit-learn.org/dev/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
# 2. https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.accuracy_score.html
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Extract logits for the final token
    final_token_logits = logits[:, -1, :]  

    # Reduce logits to predicted class indices
    predictions = final_token_logits.argmax(axis=-1)  

    # Convert token-level labels to sequence-level labels
    sequence_labels = labels[:, 0]  

    # Ensure labels are 1D
    sequence_labels = sequence_labels.squeeze()

    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(sequence_labels, predictions, average="weighted")
    accuracy = accuracy_score(sequence_labels, predictions)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [49]:
dataset = train_dataset

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load base model for LLAMA
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    num_labels=3
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA 
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="SEQ_CLS",
)

# Create training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    evaluation_strategy="epoch",
    warmup_steps=500,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Create Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    compute_metrics=compute_metrics,
    packing=packing,
)

# Training the model
trainer.train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/130 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.912270,0.046512,1.000000,0.046512,0.088889
2,2.927500,2.904527,0.046512,1.000000,0.046512,0.088889
3,2.877500,2.889863,0.046512,1.000000,0.046512,0.088889


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to contr

TrainOutput(global_step=51, training_loss=2.9093411810257854, metrics={'train_runtime': 128.9022, 'train_samples_per_second': 3.026, 'train_steps_per_second': 0.396, 'total_flos': 448487996866560.0, 'train_loss': 2.9093411810257854, 'epoch': 3.0})

In [50]:
# To save model for future use
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
# Saving trained model
trainer.model.save_pretrained("/content/drive/My Drive/llama_classifer")
trainer.save_model("/content/drive/My Drive/llama_classifer")

#**Step 5: Retrieving results**

In [52]:
# Trigger LLAMA model using the incoming news texts
classification_prompt = "Classify the following text as 'left', 'right', or 'center':\n"
sample_texts = test_df["text"].tolist()
prompted_texts = [classification_prompt + text for text in sample_texts]

# Tokenize the news texts for the model
tokenized_inputs = tokenizer(
    prompted_texts,
    truncation=True,
    padding=True,
    max_length=512,
    return_tensors="pt"
)


In [53]:
inputs = {key: val.to(model
                      .device) for key, val in tokenized_inputs.items()}

# Get logits from the model
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits  # Raw scores (logits)


In [54]:
# Rearrage predictions as shown below
predictions = torch.argmax(logits, dim=-1) 

In [55]:
print(f"Predictions List: {predictions.tolist()}")


Predictions List: [[19838, 936, 278, 1494, 9506, 408, 23329, 1066, 742, 525, 1266, 742, 470, 525, 5064, 2396, 13, 19838, 1426, 2750, 278, 352, 12929, 28913, 29892, 29899, 29984, 279, 348, 29892, 8570, 8570, 8570, 304, 278, 4749, 10387, 423, 29915, 29879, 9250, 14243, 322, 322, 10734, 278, 9892, 393, 5866, 817, 263, 10751, 310, 263, 14263, 8372, 17728, 713, 30024, 304, 9850, 29892, 20479, 756, 6471, 1827, 338, 3275, 5866, 297, 14000, 297, 22979, 29889, 633, 1509, 573, 13175, 29889, 13, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [19838, 936, 278, 1494, 9506, 408, 23329, 1066, 742, 525, 1266, 742, 470, 525, 5064, 2396, 13, 19838, 1426, 29899, 393, 7178, 2039, 1546, 278, 1153, 8818, 5700, 892, 376, 766, 1736, 22950, 310, 393, 278, 515, 451, 3447, 373, 304, 16508, 363, 304, 1044, 5565, 345, 278, 716, 297, 13, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [56]:
# Map the token IDs for labels
label_tokens = tokenizer(["left", "right", "center"], add_special_tokens=False)["input_ids"]
label_tokens = [t[0] for t in label_tokens] 
print(f"Label Tokens: {label_tokens}") 


Label Tokens: [2175, 1492, 4818]


In [57]:
final_token_logits = logits[:, -1, :] 
label_logits = final_token_logits[:, label_tokens] 


In [58]:
# Get the predicted class for each example
predictions = torch.argmax(label_logits, dim=-1)  # Shape: [batch_size]


In [59]:
# Convert predictions to labels to read results easily
label_mapping = {0: "left", 1: "right", 2: "center"}
predicted_labels = [label_mapping[p.item()] for p in predictions]


In [62]:
import pandas as pd
from google.colab import drive

# Combine predictions with actual classifications and original texts
results = [
    {
        "text": text,
        "actual_type": actual,
        "predicted_type": pred
    }
    for text, actual, pred in zip(test_df["text"], test_df["type"], predicted_labels)
]

# Print results
for i, result in enumerate(results):
    print(f"Sample {i+1}:")
    print(f"Text: {result['text']}")
    print(f"Actual Type: {result['actual_type']}")
    print(f"Predicted Type: {result['predicted_type']}")
    print()

Sample 1:
Text: The case of Rahaf Mohammed al-Qunun drawn new global attention to Saudi Arabia’s strict social rules, including a requirement that women have the permission of a male “guardian” to travel, which rights groups say can trap women and girls as prisoners of abusive families.
Actual Type: center
Predicted Type: right

Sample 2:
Text: The Post said the talks on payroll taxes were in the early stages and that officials had not decided whether to push Congress to formally approve a reduction.
Actual Type: center
Predicted Type: left

Sample 3:
Text: Colin P. Clarke has been teaching a course on terrorism and insurgency at Carnegie Mellon University in Pittsburgh for four years, and much more of his class these days is devoted to white supremacy than in the past.
Actual Type: center
Predicted Type: left

Sample 4:
Text: In Heller, the Supreme Court struck down a ban on handgun possession in the nation’s capital, ruling for the first time that the Second Amendment protects an ind

In [63]:
# Path to save the file in Google Drive
output_path = '/content/drive/My Drive/predictions_results.csv'

# Combine predictions with original texts into a DataFrame
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv(output_path, index=False)

print(f"Results saved to {output_path}")

Results saved to /content/drive/My Drive/predictions_results.csv


*end of file*